In [1]:
import tensorflow as tf
from nets import nets_factory
from preprocessing import preprocessing_factory
import reader
import model
import time
import os
import numpy as np

height = 0
width = 0
with open('./test2.jpg', 'rb') as img:
    with tf.Session().as_default() as sess:
        #if FLAGS.image_file.lower().endswith('png'):
        #    image = sess.run(tf.image.decode_png(img.read()))
        #else:
        image = sess.run(tf.image.decode_jpeg(img.read()))
        height = image.shape[0]
        width = image.shape[1]
tf.logging.info('Image size: %dx%d' % (width, height))



INFO:tensorflow:Image size: 1162x640


In [2]:
with tf.Graph().as_default():
    with tf.Session().as_default() as sess:
        image_preprocessing_fn, _ = preprocessing_factory.get_preprocessing(
            'vgg_19',
            is_training=False)
        image = reader.get_image('./test2.jpg', height, width, image_preprocessing_fn)
        image = tf.expand_dims(image, 0)
        generated = model.net(image, training=False)
        generated = tf.squeeze(generated, [0])
        saver = tf.train.Saver(tf.global_variables())
        sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
        #FLAGS.model_file = os.path.abspath(FLAGS.model_file)
        saver.restore(sess, './cubist.ckpt-done')

        start_time = time.time()
        generated = sess.run(generated)
        generated = tf.cast(generated, tf.uint8)
        end_time = time.time()
        tf.logging.info('Elapsed time: %fs' % (end_time - start_time))
        generated_file = 'generated/cubic.jpg'
        if os.path.exists('generated') is False:
            os.makedirs('generated')
        with open(generated_file, 'wb') as img:
            img.write(sess.run(tf.image.encode_jpeg(generated)))
            tf.logging.info('Done. Please check %s.' % generated_file)

INFO:tensorflow:Elapsed time: 15.299066s
INFO:tensorflow:Done. Please check generated/cubic.jpg.


In [3]:
help(tf.split)

Help on function split in module tensorflow.python.ops.array_ops:

split(value, num_or_size_splits, axis=0, num=None, name='split')
    Splits a tensor into sub tensors.
    
    If `num_or_size_splits` is a scalar, `num_split`, then splits `value` along
    dimension `axis` into `num_split` smaller tensors.
    Requires that `num_split` evenly divides `value.shape[axis]`.
    
    If `num_or_size_splits` is a tensor, `size_splits`, then splits `value` into
    `len(size_splits)` pieces. The shape of the `i`-th piece has the same size as
    the `value` except along dimension `axis` where the size is `size_splits[i]`.
    
    For example:
    
    ```python
    # 'value' is a tensor with shape [5, 30]
    # Split 'value' into 3 tensors with sizes [4, 15, 11] along dimension 1
    split0, split1, split2 = tf.split(value, [4, 15, 11], 1)
    tf.shape(split0) ==> [5, 4]
    tf.shape(split1) ==> [5, 15]
    tf.shape(split2) ==> [5, 11]
    # Split 'value' into 3 tensors along dimension 1
